In [1]:
import requests
import pandas as pd
from xml.etree import ElementTree as ET
import json
import sqlite3
import time

In [2]:
# pd.set_option('display.max_columns', None)

# 1. 사용자 인증키(본인 발급키로 대체하세요)
service_key = "E7D8NqAdhRSmPl4Xpz4znibDdyvlX6juRaTNM4Qy3k9RIDsk7camHLx7cwaTLdPKgIzIdYE%2BzOMyH0LZdVCyew%3D%3D"
pageNo = 1  # 페이지번호
numOfRows = 100  # 한 페이지 결과 수
# meet = "01"  # 01: 서울, 02: 제주, 03: 부경
rc_year = "2025"  # 경주년도(YYYY 형식)
# rc_month = "202401"  # 경주연월(YYYYMM 형식)
# rc_date = "20240101"  # 경주일자(YYYYMMDD 형식)
# rc_no = ""  # 경주번호
type = "json"  # 응답 형식 (xml, json)


# 2. API 요청 URL (예: 2024년 1월~6월 데이터, 결과는 XML)
url = (
    "http://apis.data.go.kr/B551015/API214_1/RaceDetailResult_1"
    f"?ServiceKey={service_key}"
    # f"&pageNo={pageNo}"
    f"&numOfRows={numOfRows}"
    f"&_type={type}"
)

# 3. 요청 및 응답 파싱 (XML 포맷)
response = requests.get(url)
# root = ET.fromstring(response.content)  # XML 파싱을 원할 경우 사용

content = json.loads(response.content.decode('utf-8'))
data = content['response']['body']['items']['item']
df = pd.DataFrame(data)

## To check the current working directory and save the DataFrame to a CSV file
# import os
# print(os.getcwd())
df.to_csv('C:/Users/nnopi/Downloads/horse_racing_study/df.csv', index=False, encoding='utf-8-sig')

In [3]:
# DB 연결 및 테이블 생성
conn = sqlite3.connect('C:/Users/nnopi/Downloads/race_results_2025.db')
table_name = 'race_results'

rc_year = '2025'
pageNo = 1

while True:
    url = (
        "http://apis.data.go.kr/B551015/API214_1/RaceDetailResult_1"
        f"?ServiceKey={service_key}"
        f"&pageNo={pageNo}"
        f"&numOfRows=100"
        f"&rc_year={rc_year}"
        f"&_type=json"
    )
    response = requests.get(url)
    content = json.loads(response.content.decode('utf-8'))
    # content가 string인 경우 출력
    if isinstance(content['response']['body']['items'], str):
        print("Response content is a string:", content['response']['body']['items'])
        break
    items = content['response']['body']['items']['item']
    if not items:
        break
    df = pd.DataFrame(items)
    print(df.head(2))  # 데이터 확인용 출력
    df.to_sql(table_name, conn, if_exists='append', index=False)
    pageNo += 1
    time.sleep(0.2)  # API 과다 호출 방지

conn.close()

   age ageCond  birthday  buG1fAccTime  buG1fOrd  buG2fAccTime  buG2fOrd  \
0    2      2세  20230122             0         0             0         0   
1    2      2세  20230309             0         0             0         0   

   buG3fAccTime  buG3fOrd  buG4fAccTime  ...  sj_3cOrd  sj_4cOrd  trName  \
0             0         0             0  ...         0         0     양영남   
1             0         0             0  ...         0         0     구민성   

     trNo  track  wgBudam  wgBudamBigo  wgHr  wgJk  winOdds  
0  070263   (0%)     55.5            -   0()     0        0  
1  070173   (0%)     53.5            -   0()     0        0  

[2 rows x 89 columns]
   age ageCond  birthday  buG1fAccTime  buG1fOrd  buG2fAccTime  buG2fOrd  \
0    4    연령오픈  20210409             0         0             0         0   
1    5    연령오픈  20200216             0         0             0         0   

   buG3fAccTime  buG3fOrd  buG4fAccTime  ...  sj_3cOrd  sj_4cOrd  trName  \
0             0         0   

OperationalError: table race_results has no column named weather

In [4]:
# DB 연결 및 테이블 생성
conn = sqlite3.connect('C:/Users/nnopi/Downloads/race_results_2025.db')
cursor = conn.cursor()
table_name = 'race_results'

try:
    # Drop existing table to ensure clean schema
    cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    conn.commit()
    
    rc_year = '2025'  # Keep original year
    pageNo = 1
    first_iteration = True

    while True:
        url = (
            "http://apis.data.go.kr/B551015/API214_1/RaceDetailResult_1"
            f"?ServiceKey={service_key}"
            f"&pageNo={pageNo}"
            f"&numOfRows=100"
            f"&rc_year={rc_year}"
            f"&_type=json"
        )
        
        response = requests.get(url)
        content = json.loads(response.content.decode('utf-8'))
        
        # Debug output on first iteration
        if first_iteration:
            print(f"Total Count: {content['response']['body'].get('totalCount', 0)}")
        
        if isinstance(content['response']['body']['items'], str):
            print("Response content is a string:", content['response']['body']['items'])
            break
            
        items = content['response']['body']['items']['item']
        if not items:
            break
            
        # Convert to list if single item
        if isinstance(items, dict):
            items = [items]
            
        df = pd.DataFrame(items)
        
        # Create table with correct schema on first iteration
        if first_iteration:
            df.head(0).to_sql(table_name, conn, if_exists='replace', index=False)
            first_iteration = False
            print(f"Created table with columns: {', '.join(df.columns)}")
        
        print(f"Processing page {pageNo}, records: {len(df)}")
        df.to_sql(table_name, conn, if_exists='append', index=False)
        conn.commit()  # Commit after each batch
        
        pageNo += 1
        time.sleep(0.2)

except Exception as e:
    print(f"Error details: {str(e)}")
    conn.rollback()
finally:
    conn.close()

print("Data collection completed")

Total Count: 15908
Created table with columns: age, ageCond, birthday, buG1fAccTime, buG1fOrd, buG2fAccTime, buG2fOrd, buG3fAccTime, buG3fOrd, buG4fAccTime, buG4fOrd, buG6fAccTime, buG6fOrd, buG8fAccTime, buG8fOrd, buS1fAccTime, buS1fOrd, buS1fTime, bu_10_8fTime, bu_1fGTime, bu_2fGTime, bu_3fGTime, bu_4_2fTime, bu_6_4fTime, bu_8_6fTime, budam, buga1, buga2, buga3, chaksun1, chaksun2, chaksun3, chaksun4, chaksun5, chulNo, diffUnit, hrName, hrNo, hrTool, ilsu, jeG1fTime, jeG3fTime, jeS1fTime, je_1cTime, je_2cTime, je_3cTime, je_4cTime, jkName, jkNo, meet, name, ord, ordBigo, owName, owNo, plcOdds, prizeCond, rank, rankRise, rating, rcDate, rcDay, rcDist, rcName, rcNo, rcTime, seG1fAccTime, seG3fAccTime, seS1fAccTime, se_1cAccTime, se_2cAccTime, se_3cAccTime, se_4cAccTime, sex, sjG1fOrd, sjG3fOrd, sjS1fOrd, sj_1cOrd, sj_2cOrd, sj_3cOrd, sj_4cOrd, trName, trNo, track, wgBudam, wgBudamBigo, wgHr, wgJk, winOdds
Processing page 1, records: 100
Processing page 2, records: 100
Processing page 3